The following code cell ensures that the underlying `.py` file gets automatically reloaded after you save any changes.  

In [13]:
%load_ext autoreload
%autoreload 2

In [1]:
import pandas as pd    
from model import initialize_model, call_llm
import time



/Users/jasonchung/Downloads/nlp/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

def preprocess(file_path):
    data = pd.read_json(path_or_buf=file_path, lines=True)
    ids = set()
    rows_to_be_dropped = []
    for i in range(len(data)):
        row = data.iloc[i]
        if row.unique_id in ids:
            rows_to_be_dropped.append(i)
        else:
            ids.add(row.unique_id)
    data = data.drop(rows_to_be_dropped)
    return data

In [3]:
data = preprocess("./data/train.jsonl")

In [ ]:
first_partition = data[:1365]
second_partition = data[1365:2730]
third_partition = data[2730:4095]
fourth_partition = data[4095:5460]
fifth_partition = data[5460:6825]
sixth_partition = data[6825:]

In [ ]:
labels = []
reasonings = []
raw_output = []
ids = []
client, _ = initialize_model("")
last_time = time.time()

# Change len(data) to len(your_partition)
for i in range(len(data)):
    if i > 0 and i % 60 == 0:
        print(i)
        current_time = time.time()
        time.sleep(max(0, 63 - (current_time - last_time)))
        last_time = time.time()

    # Change data to your partition
    current_data = data.iloc[i]
    ids.append(current_data.unique_id)
    classification, reasoning = call_llm(client, "", current_data.string, current_data.sectionName)
    raw_output.append(classification)
    labels.append(classification)
    reasonings.append(reasoning)
    

id='no4BXWV-4Yz4kd-9285e0c83a559c32' object=<ObjectType.ChatCompletion: 'chat.completion'> created=1743319579 model='meta-llama/Llama-3.3-70B-Instruct-Turbo-Free' choices=[ChatCompletionChoicesData(index=0, logprobs=None, seed=5790044909235756000, finish_reason=<FinishReason.StopSequence: 'stop'>, message=ChatCompletionMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='{"classification": "background", "reasoning": "The text is classified as background because it provides context and introduces the research topic by discussing the current state of knowledge and the unclear aspects of frataxin\'s interaction with Fe-S cluster biosynthesis components, which is typical of an introduction section in a scientific paper."}', tool_calls=[]))] prompt=[] usage=UsageData(prompt_tokens=215, completion_tokens=65, total_tokens=280)
id='no4Bb8M-4Yz4kd-9285e1145b4f9c32' object=<ObjectType.ChatCompletion: 'chat.completion'> created=1743319591 model='meta-llama/Llama-3.3-70B-Instruct-Turbo-Free

KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame(zip(ids, reasonings), columns=["id", "reasoning"])
df.to_csv("your_partition.csv")